# RAG using MemoryDB

[Retrieval Augmented Generation](https://arxiv.org/abs/2005.11401) is a process that combines retrieval-based models and generative models to enhance natural language generation by retrieving relevant information and incorporating it into the generation process. 

In this lab we are going to be writing a simple RAG application code that allows user to ask questions about various wines so they can make a purchasing decision. We will use the vector search capability with MemoryDB to retrieve the best matching wine reviews and provide that to LLM for answering user's questions.

## 1. Lab Pre-requisites

#### a. Download and install python dependencies

For this notebook we require the use of a few libraries. We'll use the Python clients for Redis and SageMaker, and Python frameworks for text embeddings.

In [ ]:
!pip install redis pandas numpy sentence-transformers boto3 ipywidgets --quiet

### 3. Import libraries & initialize resource information
The line below will import all the relevant libraries and modules used in this notebook.

In [ ]:
import redis
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import boto3
import json
from IPython.display import display, HTML
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

#### Get CloudFormation stack output variables

We have preconfigured a few resources by creating a cloudformation stack in the account. Information of these resources will be used within this lab. We are going to load some of the information variables here.

You can ignore any "PythonDeprecationWarning" warnings.

In [ ]:
# Create a Boto3 session
session = boto3.Session()

# Get the current region
region = session.region_name

cfn = boto3.client('cloudformation')

# Method to obtain output variables from Cloudformation stack. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "genai-data-foundation-workshop"

outputs = get_cfn_outputs(cloudformation_stack_name)
memorydb_endpoint = outputs['MemoryDBClusterEndpoint']
sagemaker_notebook_url = outputs['SageMakerNotebookURL']

# We will just print all the variables so you can easily copy if needed.
outputs

## 3. Prepare data
Below is the code that loads dataset of wine reviews, we'll use this data set to recommend wines that resemble the user provided description.

#### Sampling subset of the records to load into opensearch quickly
Since the data is composed of 129,000 records, it could take some time to convert them into vectors and load them in a vector store. Therefore, we will take a subset (300 records) of our data. We will add a variable called record_id which corresponds to the index of the record

In [ ]:
url = "https://raw.githubusercontent.com/davestroud/Wine/master/winemag-data-130k-v2.json"
df = pd.read_json(url)
df_sample = df.sample(300,random_state=37).reset_index()
df_sample['record_id'] = range(1, len(df_sample) + 1)
df_sample[:5]

## 4. Create a connection with MemoryDB cluster.
Next, we'll use Python API to set up connection with MemoryDB Cluster.

In [ ]:
# Connect to Redis
redis_conn = redis.RedisCluster(
    host=memorydb_endpoint,
    port=6379,
    ssl=True,
    decode_responses=True
)

## 5. Using Amazon Bedrock titan embedding to convert text to vectors
Amazon Bedrock offers Amazon titan embedding v2 model that generates vector embeddings for text. This model will be used as our primary model for embeddings.

#### Helper method to invoke Titan embedding model in Amazon Bedrock
Creating a helper method in python to invoke Amazon Titan embedding model from Amazon Bedrock to generate embeddings. We will update `df_sample` data frame and add a new column called `embedding` in it. Once this cell is executed, our data frame will be ready to load into MemoryDB.

In [ ]:
import boto3
import pandas as pd
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


bedrock_client = boto3.client(
    "bedrock-runtime", 
    region, 
    endpoint_url=f"https://bedrock-runtime.{region}.amazonaws.com"
)


def add_embeddings_to_df(df, text_column):

    # Create an empty list to store embeddings
    embeddings = []

    # Iterate over the text in the specified column
    for text in df[text_column]:
        embedding = embed_phrase(text)
        embeddings.append(embedding)
        

    # Add the embeddings as a new column to the DataFrame
    df['embedding'] = embeddings

    return df

def embed_phrase( text ):
        
    model_id = "amazon.titan-embed-text-v2:0"  # 
    accept = "application/json"
    contentType = "application/json"

    # Prepare the request payload
    request_payload = json.dumps({"inputText": text})


    response = bedrock_client.invoke_model(body=request_payload, modelId=model_id, accept=accept, contentType=contentType)

    # Extract the embedding from the response
    response_body = json.loads(response.get('body').read())


    # Append the embedding to the list
    embedding = response_body.get("embedding")
    return embedding

df_sample = add_embeddings_to_df(df_sample, 'description')

df_sample[:5]

#### Let's try to create an embedding of a simple input text
You can see its an array of floating point numbers. While it does not make sense to human eye/brain, this array of numbers captures the semantics and knowledge of the text and that can be later used to compare two different text blocks. This method will be used to convert our query to a vector representation.

In [ ]:
## Create an vector embedding for input text
input_text = "A wine that pairs well with turkey breast?"

embedding = embed_phrase(input_text)

#printing text and embedding

print(f"{input_text=}")

#only printing first 10 dimensions of the 1024 dimension vector 
print(f"{embedding[:10]=}")

## 7. Create a index in Amazon MemoryDB 
Whereas we previously created an index with 2-3 fields, this time we'll define the index with multiple fields: the vectorization of the `description` field, and all others present within the dataset.

To create the index, we first define the index in JSON, then use the redis client we initiated ealier to create the vector index in MemoryDB.

In [ ]:
# define schema
schema = (
    TextField("description"),
    TextField("winery"),
    TextField("country"),
    TextField("designation"),
    TextField("variety"),
    TextField("points"),
    TextField("wine_name"),
    VectorField("embedding", "FLAT", {"TYPE": "FLOAT32", "DIM": 1024, "DISTANCE_METRIC": "COSINE"})
)

Using the above index definition, we now need to create the index in Amazon MemoryDB. Running this cell will recreate the index if you have already executed this notebook.

In [ ]:
redis_conn.ft('wine_index').create_index(fields=schema, definition=IndexDefinition(prefix=['doc:'], index_type=IndexType.HASH))

Let's verify the created index information

In [ ]:
redis_conn.ft('wine_index').info()

## 8. Load the raw data into the Index
Next, let's load the wine review data and embedding into the index we've just created. Notice that we will store our embedding in `description_vector` field which will later be used for KNN search

In [ ]:
def load_vectors(client, df):
    p = client.pipeline(transaction=False)
    for i, row in df.iterrows():
        key = f"doc:{row['record_id']}"
        data = {
            "description": row['description'] or '',
            "winery": row['winery'] or '',
            "country": row['country'] or '',
            "designation": row['designation'] or '',
            "variety": row['variety'] or '',
            "points": row['points'] or '',
            "wine_name": row['title'] or '',            
            "embedding": np.array(row['embedding'], dtype=np.float32).tobytes()
        }
        p.hset(key, mapping=data)
    p.execute()

# loading data
load_vectors(redis_conn, df_sample)

To validate the load, we'll query the number of documents number in the index. We should have 300 hits in the index, or however many was specified earlier in sampling.

In [ ]:
result = redis_conn.ft('wine_index').info()
print("Records found: %d." % result['num_indexed_vectors'])

## 9. Search vector

Now we can define a helper function to execute the search query for us to find a wine whose review most closely matches the requested description. `search_vector` embeds the search phrase, searches the index for the closest matching vector, and returns the top result.


In [ ]:
def search_vectors(client, query, k=3):
    query_vector = np.array(embed_phrase(query), dtype=np.float32).tobytes()

    search_query = f"*=>[KNN {k} @embedding $query_vector AS score]"
    params = {"query_vector": query_vector}
    result = client.ft('wine_index').search(
        Query(search_query)
        .sort_by("score")
        .return_fields("description", "winery", "country", "designation", "variety", "points", "wine_name")
        .dialect(2),
        query_params=params
    )
    
    results = []
    for doc in result.docs:
        results.append({
            "description": doc.description,
            "winery": doc.winery,
            "country": doc.country,
            "designation": doc.designation,
            "variety": doc.variety,
            "points": doc.points,
            "wine_name": doc.wine_name            
        })
    return results

Use the semantic search to get similar records with the sample question

In [ ]:
query = "Best Australian wine that goes great with steak?"

results = search_vectors(redis_conn, query)
print(results)

## 10. Prepare a method to call Amazon Bedrock - Anthropic Claude Sonnet model

Now we will define a function to call LLM to answer user's question. As LLM is trained with static data, and it does not have our wine review knowledge. While it may be able to answer, it may not be an answer that a business prefers. For example. in our case, we would not want to recommend a wine that we do not stock. So the recommendation has to be one of the wines from our collection i.e. 300 reviews that we loaded. 

After defining this function we will call it to see how LLM answers questions without the wine review data.

In [ ]:
def query_llm(system, user_question):
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10000,
        "system": system,
        "messages": [{"role": "user", "content": user_question}]
    })
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=payload.encode('utf-8'),
        accept="application/json",
        contentType="application/json"
    )
    response_body = json.loads(response['body'].read())
    return response_body['content'][0]['text']


Let's check the generated result for a wine recommendation. It may not be one of the wine that we stock.

In [ ]:
def query_llm_without_rag(question):
    
    #Claude model has 2 parts of the prompt. System prompt guides the model what role to play
    system_prompt = f"You are a sommelier that uses their vast knowledge of wine to make great recommendations people will enjoy."
    
    #User prompt is the engineer prompt that has the context that model should reference to answer questions
    user_prompt = (
        f" As a sommelier, you must include the wine variety, the country of origin, "
        f"and a colorful description relating to the customer question."
        f"\n Customer question: {question}"
        f"\n Please provide name of the wine at the end of the answer, in a new line, in format Wine name: <wine name>"
    )
    return query_llm(system_prompt, user_prompt)


question_on_wine="Best Australian wine that goes great with steak?"

print(f"The recommened wine from LLM without RAG: \n{query_llm_without_rag(question_on_wine)}\n")

#### Testing for hallucination. 
Let's copy the wine name from the last line and past it in the question variable below to see if we have this wine in our stock. Please review the list of wines that are returned. They may be from portugal but not exactly the one we have been recommended by the model.

__Note:__ If you do not see the same wine name in the `wine_name` variable below, you should replace it so we can verify that the wine recommended is not in our index. 

In [ ]:
wine_name = "Penfolds Bin 389 Kalimna Shiraz"
example_request = query_llm_without_rag(wine_name)
print("Matching wine records in our reviews:")
print(json.dumps(example_request, indent=4))

## 11. Retrieval Augmented Generation
---
To resolve LLM hallunination problem, we can more context to LLM so that LLM can use context information to fine the model and generated factual result. RAG is one of the solution to the LLM hallucination. 


#### Create a prompt for the LLM using the search results from MemoryDB (RAG)

We will be using the Anthropic Sonnet model with one-shot prompting technique. Within instructions to the model in the prompt, we will provide a sample wine review and how model should use to answer user's question. At the end of the prompt wine reviews retrieved from MemoryDB will be included for model to use. 

Before querying the model, the below function `generate_rag_based_system_prompt` is used to put together user prompt. The function takes in an input string to search the MemoryDB cluster for a matching wine, then compose the user prompt for LLM. 

System prompt defines the role that LLM plays.

User prompt contains the instructions and the context information that LLM model uses to answer user's question.

The prompt is in the following format:

**SYSTEM PROMPT:**

```
You are a sommelier that uses their vast knowledge of wine to make great recommendations people will enjoy. 
```


**USER PROMPT**
```
As a sommelier, you must include the wine variety, the country of origin, and a colorful description relating to the user's question.

Data:{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}

Recommendation:I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator.

Data: {retrieved_documents}

Question from the user as is
```



### package the prompt and query the LLM
We will create a final function to query the LLM with the prompt. `query_llm_with_rag` is a function that calls LLM in a RAG.

`query_llm_with_rag` combines everything we've done in this module. It does all of the following:
- searches the vector index for the relevant wine with "description vector"
- generate an LLM prompt from the search results
- queriy the LLM with RAG for a response

In [ ]:
def query_llm_with_rag(user_question):
    retrieved_documents = search_vectors(redis_conn, user_question, 10)
    one_shot_description_example = "{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}"
    one_shot_response_example = "I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator."
    system_prompt = "You are a sommelier that uses vast knowledge of wine to make great recommendations people will enjoy."
    user_prompt = (
        f"As a sommelier, you must include the wine variety, the country of origin, and a colorful description relating to the user question. You are must pick a wine in \"Wine data\" section only, one that matches best the customer question. Do not recommend any wine outside of the wine data provided and you must not suggest further. You don't necessarily have to pick the top rated wine if its not best suited for customer question.\n"
        f"Example Wine data: {one_shot_description_example} \n Example Recommendation: {one_shot_response_example} \n"
        f"Wine data: {retrieved_documents}\n"
        f"Customer Question: {user_question}\n"
    )
    response = query_llm(system_prompt, user_prompt)
    return response

#### And finally, let's call the function and get a wine recommendation.

In [ ]:
question_on_wine="Best Australian wine that goes great with steak?"
recommendation = query_llm_with_rag(question_on_wine)
print(recommendation)

#### Let's change it to Italian wine - it should produce a matching result.
We will call the same method again to see if there is an italian wine in our catalog.

In [ ]:
question_on_wine="Best Italian wine that goes great with steak?"
recommendation = query_llm_with_rag(question_on_wine)
print(recommendation)

You might notice that we asked for Australian wines that goes well with steak and we do not have any such wine in our collection. Therefore the model politely excuses. You may change the question and see how LLM recommends a wine from our select list that best suites your question.

### Conclusion
In this lab you built a simple wine recommendation chatbot. In this particular lab you used Amazon Bedrock titan v2 model to create vector embedding for our data. Then we loaded this data in OpenSearch index with `description_vector` field. At search time, we used Amazon Titan v2 model again to convert our query question into vector embedding and used semantic search to retrieve results. These results were then passed on to Anthropic Claude Sonnet 3 model which was able to recommend us a wine from within our catalog.

## Lab finished - you may now go back to lab instructions section